In [6]:
import pandas as pd
from tqdm.notebook import tqdm
from src.gbrt import prior_prob
from src.utils import wikipedia_search, ddg_search, google_kb_search

In [7]:
# Get all unique mentions and their tag
mentions = []
for doc in range(1, 1394):
    df = pd.read_csv(f"./data/aida/candidates/{doc}.csv")
    doc_mentions = df[['mention', 'tag']].drop_duplicates().to_numpy().tolist()
    for i in doc_mentions:
        if i not in mentions:
            mentions.append(i)

In [ ]:
google_kg_results = []
ddg_results  = []
wiki_results = []
comb_results = []
for mention, tag in tqdm(mentions):
    d_res = ddg_search(mention)
    w_res = wikipedia_search(mention, limit=50)
    g_res = google_kb_search(mention, limit=50)
    c_res = list(dict.fromkeys(d_res + w_res + g_res))
    if tag in d_res:
        ddg_results.append(d_res.index(tag) + 1)
    if tag in w_res:
        wiki_results.append(w_res.index(tag) + 1)
    if tag in g_res:
        google_kg_results.append(g_res.index(tag) + 1)
    if tag in c_res:
        comb_results.append(c_res.index(tag) + 1)

In [5]:
avg = lambda x : sum(x)/len(x)
acc = lambda x : round(len(x)/len(mentions), 4) * 100

print(f"Google KG  : {acc(google_kg_results)}\tavg pos: {avg(google_kg_results)}")
print(f"Wikipedia  : {acc(wiki_results)     }\tavg pos: {avg(wiki_results)     }")
print(f"DuckDuckGo : {acc(ddg_results)      }\tavg pos: {avg(ddg_results)      }")
print(f"Combined   : {acc(comb_results)     }\tavg pos: {avg(comb_results)     }")

Google KG  : 78.83	avg pos: 2.3195126386615748
Wikipedia  : 82.24000000000001	avg pos: 2.3561094648771133
DuckDuckGo : 77.14	avg pos: 8.15108715852072
Combined   : 90.42	avg pos: 10.57863031071655


In [16]:
res = 0
for mention, tag in tqdm(mentions):
    cands = list(prior_prob.get(mention.lower(), {}).keys())
    if tag.replace('_', ' ') in cands:
        res += 1

print(res/len(mentions))

  0%|          | 0/6976 [00:00<?, ?it/s]

0.8990825688073395
